<a href="https://colab.research.google.com/github/luiseduaardo/NeuroPrev/blob/main/infer%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/luiseduaardo/NeuroPrev.git
%cd NeuroPrev

Cloning into 'NeuroPrev'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 39 (delta 11), reused 18 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 631.45 KiB | 6.07 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/NeuroPrev


In [1]:
!pip -q install xgboost scikit-learn pandas numpy joblib

In [2]:
import json
import joblib
import pandas as pd
import numpy as np

# Carrega modelo
model = joblib.load("weights/xgb_model.pkl")

# Carrega artefatos (median_bmi, threshold e feature_columns)
with open("weights/xgb_artefatos.json", "r") as f:
    artefatos = json.load(f)

median_bmi = artefatos["median_bmi"]
threshold = artefatos["threshold"]
feature_columns = artefatos["feature_columns"]

print("✅ Modelo carregado")
print("median_bmi:", median_bmi)
print("threshold:", threshold)
print("n_features:", len(feature_columns))


✅ Modelo carregado
median_bmi: 28.0
threshold: 0.4
n_features: 21


In [3]:
#preprocessamento pra inferencia
def preprocess_for_inference(df_raw: pd.DataFrame, median_bmi: float, feature_columns: list[str]) -> pd.DataFrame:
    df = df_raw.copy()

    if "stroke" in df.columns:
        df = df.drop("stroke", axis=1)
    if "id" in df.columns:
        df = df.drop("id", axis=1)

    # imputação bmi com valor dos weights
    if "bmi" in df.columns:
        df["bmi"] = df["bmi"].fillna(median_bmi)

    # one-hot encoding
    X = pd.get_dummies(df)

    # alinhar colunas 
    X = X.reindex(columns=feature_columns, fill_value=0)

    return X


In [4]:
df_input = pd.read_csv("dataset/healthcare-dataset-stroke-data.csv")
df_input.head()


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [5]:
X_input = preprocess_for_inference(df_input, median_bmi, feature_columns)

proba = model.predict_proba(X_input)[:, 1]
pred = (proba >= threshold).astype(int)

saida = df_input.copy()
saida["prob_avc"] = proba
saida["pred_avc"] = pred

saida.head(10)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,prob_avc,pred_avc
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1,0.752754,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1,0.667139,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1,0.781081,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1,0.549362,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1,0.781438,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1,0.809770,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1,0.795403,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1,0.567204,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1,0.611162,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1,0.716808,1


In [6]:
saida.to_csv("predicoes_avc.csv", index=False)
print("✅ Arquivo salvo: predicoes_avc.csv")


✅ Arquivo salvo: predicoes_avc.csv


In [2]:
import pandas as pd
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
modelo = joblib.load('weights/modelo_avc_final.pkl')
df = pd.read_csv('dataset/healthcare-dataset-stroke-data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'weights/modelo_avc_final.pkl'

In [ ]:
df = df.drop('id', axis = 1) #remove coluna id
df['bmi'] = df['bmi'].fillna(df['bmi'].median()) #preenche valores vazios de bmi com mediana
df = pd.get_dummies(df, columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'])

In [ ]:
bool_cols = df.select_dtypes(include = ['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)

In [ ]:
X = df.drop('stroke', axis = 1)
y = df['stroke']

In [ ]:
#treino
probabilidades = modelo.predict_proba(X)[:, 1]

limiar_final = 0.20
previsoes_finais = (probabilidades >= limiar_final).astype(int)

print(classification_report(y, previsoes_finais))

In [ ]:
cm = confusion_matrix(y, previsoes_finais)
plt.figure(figsize = (8, 6))
sns.heatmap(cm, annot =True, fmt ='d', cmap = 'Blues', cbar = False,
            xticklabels = ['Pred: Saudável', 'Pred: AVC'],
            yticklabels = ['Real: Saudável', 'Real: AVC'])

plt.title('Matriz de Confusão - Random Forest (Limiar 20%)', fontsize = 14)
plt.ylabel('Realidade')
plt.xlabel('Previsão')
plt.show()